# Inadimpletes - Hackday 3° Edição

In [ ]:
#!pip install sweetviz xgboost category_encoders

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import sweetviz
import pickle

from sklearn.metrics       import mean_absolute_error, mean_squared_error, f1_score
from sklearn.ensemble      import RandomForestClassifier
from sklearn.linear_model  import LinearRegression, Lasso
from sklearn.preprocessing import StandardScaler, RobustScaler, MinMaxScaler, LabelEncoder
import sklearn.model_selection as ms
from xgboost                 import XGBClassifier
from category_encoders       import TargetEncoder

%matplotlib inline
sns.set_theme()
pd.set_option('display.float_format', '{:.2f}'.format)
pd.set_option('display.max_columns', 500)
plt.rcParams['figure.figsize'] = [25, 12]
plt.rcParams['font.size'] = 16

## Data Ingestion

In [2]:
df_train = pd.read_csv('data/train.csv')
df_train.head()

,id_cliente,idade,saldo_atual,divida_atual,renda_anual,valor_em_investimentos,taxa_utilizacao_credito,num_emprestimos,num_contas_bancarias,num_cartoes_credito,dias_atraso_dt_venc,num_pgtos_atrasados,num_consultas_credito,taxa_juros,investe_exterior,pessoa_polit_exp,limite_adicional
0,1767,21,278.17,2577.05,24196.90,104.31,31.04,6,5,7,21,14,9,15,Não,Não,Negar
1,11920,40,268.87,2465.39,19227.38,69.86,36.92,5,8,5,40,23,10,18,Não,Não,Negar
2,8910,36,446.64,1055.29,42822.28,134.20,34.56,0,3,6,26,13,3,15,Sim,Não,Negar
3,4964,58,321.14,703.05,51786.83,297.35,31.49,0,3,7,12,7,2,1,Sim,Não,Negar
4,10100,35,428.72,891.29,44626.85,134.20,28.03,2,8,7,24,10,8,20,Sim,Não,Negar


## Data understanding

## Data dimension

In [ ]:
df_train.shape

## Class distribution

In [ ]:
df_train['limite_adicional'].value_counts(normalize=True)*100

## Data types

In [ ]:
df_train.dtypes

## NAN?

In [ ]:
df_train.isna().sum()

## Descriptive Statistics

In [ ]:
df_train.describe(include='object').T.sort_values(by='unique', ascending=False)

In [ ]:
df_train.describe().T

## Data Cleaning

In [3]:
df2 = df_train.copy()

In [ ]:
def filter_outliers(df, col, factor=2):
    q3 = df[col].quantile(0.75)
    q1 = df[col].quantile(0.25)

    iqr = q3 - q1
    max_bouderie = q3 + iqr * factor
    min_bouderie = q1 - iqr * factor

    return df.loc[(df[col] >= min_bouderie) & (df[col] <= max_bouderie), :]

In [ ]:
df2 = filter_outliers(df2, 'idade')
df2 = filter_outliers(df2, 'valor_em_investimentos')
df2 = filter_outliers(df2, 'taxa_juros')
df2 = filter_outliers(df2, 'renda_anual')
df2 = filter_outliers(df2, 'num_emprestimos')
df2 = filter_outliers(df2, 'num_cartoes_credito')
df2 = filter_outliers(df2, 'num_contas_bancarias')
df2 = filter_outliers(df2, 'num_consultas_credito')

## Remove useless features

In [4]:
cols_remove = ['id_cliente']
df2 = df2.drop(columns=cols_remove)

## Dataset Split

In [23]:
df_pp = df2.copy()

In [24]:
X = df_pp.drop(columns=['limite_adicional'])
y = df_pp['limite_adicional']

X_train, X_test, y_train, y_test = ms.train_test_split(X, y, stratify=y, test_size=0.2, random_state=42)

In [25]:
# df3 será usado para splitar Treino e Validação
df3 = pd.concat([X_train, y_train], axis = 1)



# Feature engeneering

In [26]:
def filter_outliers(df, col, factor=2):
    q3 = df[col].quantile(0.75)
    q1 = df[col].quantile(0.25)

    iqr = q3 - q1
    max_bouderie = q3 + iqr * factor
    min_bouderie = q1 - iqr * factor

    return df.loc[(df[col] >= min_bouderie) & (df[col] <= max_bouderie), :]

In [27]:
df3 = filter_outliers(df3, 'idade')
df3 = filter_outliers(df3, 'valor_em_investimentos')
df3 = filter_outliers(df3, 'taxa_juros')
df3 = filter_outliers(df3, 'renda_anual')
df3 = filter_outliers(df3, 'num_emprestimos')
df3 = filter_outliers(df3, 'num_cartoes_credito')
df3 = filter_outliers(df3, 'num_contas_bancarias')
df3 = filter_outliers(df3, 'num_consultas_credito')

In [28]:
# Dividir dataset entre Treino (90%) e Validação (10%)
X = df3.drop(['limite_adicional'], axis=1).copy()
y = df3['limite_adicional'].copy()

x_train, x_val, y_train, y_val = ms.train_test_split(X, y, stratify=y, test_size=0.10, random_state = 42)

# Data Preparation

In [29]:
# Encodar features categóricas de treino

## Normalizacao
ss = StandardScaler()

## Frequency Encoding
x_train['investe_exterior'] = x_train['investe_exterior'].map(lambda x: 0.76 if x == 0.15 else 0.24)
x_train['pessoa_polit_exp'] = x_train['pessoa_polit_exp'].map(lambda x: 0.94 if x == 0.16 else 0.06)

## Rescaling
rs = RobustScaler()
mms = MinMaxScaler()

## Robust Scaler
x_train['idade'] = rs.fit_transform( x_train[['idade']].values )
x_train['saldo_atual'] = rs.fit_transform( x_train[['saldo_atual']].values )
x_train['renda_anual'] = rs.fit_transform( x_train[['renda_anual']].values )
x_train['valor_em_investimentos'] = rs.fit_transform( x_train[['valor_em_investimentos']].values )
x_train['num_emprestimos'] = rs.fit_transform( x_train[['num_emprestimos']].values )
x_train['num_contas_bancarias'] = rs.fit_transform( x_train[['num_contas_bancarias']].values )
x_train['num_cartoes_credito'] = rs.fit_transform( x_train[['num_cartoes_credito']].values )
# x_train['num_pgtos_atrasados'] = rs.fit_transform( x_train[['num_pgtos_atrasados']].values )
x_train['num_consultas_credito'] = rs.fit_transform( x_train[['num_consultas_credito']].values )
x_train['taxa_juros'] = rs.fit_transform( x_train[['taxa_juros']].values )
## Min-max scaler
x_train['dias_atraso_dt_venc'] = mms.fit_transform( x_train[['dias_atraso_dt_venc']].values )
x_train['divida_atual'] = mms.fit_transform( x_train[['divida_atual']].values )
x_train['taxa_utilizacao_credito'] = ss.fit_transform( x_train[['taxa_utilizacao_credito']].values )

# Label Enconder
le = LabelEncoder()
df5['comprometimento_renda'] = le.fit_transform(df5['comprometimento_renda'])

pg_atrasados_dict = {'Excelente': 4, 'Bom': 3, 'Regular': 2, 'Ruim': 1}
df5['num_pgtos_atrasados'] = df5['num_pgtos_atrasados'].map(pg_atrasados_dict)


# Encodar var resposta de treino
d_class = {
    'Negar': 0,
    'Conceder': 1
}

y_train = y_train.map(d_class)

In [31]:
# Encodar features categóricas de treino

## Normalizacao
ss = StandardScaler()

## Frequency Encoding
x_val['investe_exterior'] = x_val['investe_exterior'].map(lambda x: 0.76 if x == 0.15 else 0.24)
x_val['pessoa_polit_exp'] = x_val['pessoa_polit_exp'].map(lambda x: 0.94 if x == 0.16 else 0.06)

## Rescaling
rs = RobustScaler()
mms = MinMaxScaler()

## Robust Scaler
x_val['idade'] = rs.fit_transform( x_val[['idade']].values )
x_val['saldo_atual'] = rs.fit_transform( x_val[['saldo_atual']].values )
x_val['renda_anual'] = rs.fit_transform( x_val[['renda_anual']].values )
x_val['valor_em_investimentos'] = rs.fit_transform( x_val[['valor_em_investimentos']].values )
x_val['num_emprestimos'] = rs.fit_transform( x_val[['num_emprestimos']].values )
x_val['num_contas_bancarias'] = rs.fit_transform( x_val[['num_contas_bancarias']].values )
x_val['num_cartoes_credito'] = rs.fit_transform( x_val[['num_cartoes_credito']].values )
x_val['num_pgtos_atrasados'] = rs.fit_transform( x_val[['num_pgtos_atrasados']].values )
x_val['num_consultas_credito'] = rs.fit_transform( x_val[['num_consultas_credito']].values )
x_val['taxa_juros'] = rs.fit_transform( x_val[['taxa_juros']].values )
## Min-max scaler
x_val['dias_atraso_dt_venc'] = mms.fit_transform( x_val[['dias_atraso_dt_venc']].values )
x_val['divida_atual'] = mms.fit_transform( x_val[['divida_atual']].values )
x_val['taxa_utilizacao_credito'] = ss.fit_transform( x_val[['taxa_utilizacao_credito']].values )

# Encodar var resposta de treino
d_class = {
    'Negar': 0,
    'Conceder': 1
}

y_val = y_val.map(d_class)

## Modeling & Evaluation

In [32]:
# Aplicação de modelos 
# Treinar XGBoost Classifier
# Model definition
model_xgboost = XGBClassifier(random_state=42)
# Model training
model_xgboost.fit(x_train, y_train)
# Model prediction
yhat_xgboost = model_xgboost.predict(x_val)

In [33]:
# Avaliação de modelos 
# Validar modelo com F1 Score (dados validação)
print(f"XGBoost - Média ponderada com F1 Score (dados validação): {round(f1_score(y_val, yhat_xgboost, average='weighted') ,3)}") #binary=default

XGBoost - Média ponderada com F1 Score (dados validação): 0.864


In [34]:
# Validar modelo com F1 Score, no método cross validation (dados validação) 
print(f"K-folds: {ms.cross_val_score(model_xgboost, x_val, y_val, scoring='f1', cv = 10)}") #'F1' métrica F1 Score p/ classificação binary
print(f"XGBoost - Média ponderada com F1 Score (dados validação): {round(ms.cross_val_score(model_xgboost, x_val, y_val, scoring='f1_weighted', cv = 10).mean() ,3)}")
# Performance manteve-se próxima com cross validation

K-folds: [0.625      0.28571429 0.84210526 0.44444444 0.35294118 0.6
 0.35294118 0.52631579 0.55555556 0.75      ]
XGBoost - Média ponderada com F1 Score (dados validação): 0.862


## All dataset

In [35]:
# Testar modelos
# Antes, juntar dados de treino + validação, formando um novo dataset de treino.
x_train_full = pd.concat([x_train, x_val])
y_train_full = pd.concat([y_train, y_val])

In [36]:
# Encodar features categóricas de treino

## Normalizacao
ss = StandardScaler()

## Frequency Encoding
X_test['investe_exterior'] = X_test['investe_exterior'].map(lambda x: 0.76 if x == 0.15 else 0.24)
X_test['pessoa_polit_exp'] = X_test['pessoa_polit_exp'].map(lambda x: 0.94 if x == 0.16 else 0.06)

## Rescaling
rs = RobustScaler()
mms = MinMaxScaler()

## Robust Scaler
X_test['idade'] = rs.fit_transform( X_test[['idade']].values )
X_test['saldo_atual'] = rs.fit_transform( X_test[['saldo_atual']].values )
X_test['renda_anual'] = rs.fit_transform( X_test[['renda_anual']].values )
X_test['valor_em_investimentos'] = rs.fit_transform( X_test[['valor_em_investimentos']].values )
X_test['num_emprestimos'] = rs.fit_transform( X_test[['num_emprestimos']].values )
X_test['num_contas_bancarias'] = rs.fit_transform( X_test[['num_contas_bancarias']].values )
X_test['num_cartoes_credito'] = rs.fit_transform( X_test[['num_cartoes_credito']].values )
X_test['num_pgtos_atrasados'] = rs.fit_transform( X_test[['num_pgtos_atrasados']].values )
X_test['num_consultas_credito'] = rs.fit_transform( X_test[['num_consultas_credito']].values )
X_test['taxa_juros'] = rs.fit_transform( X_test[['taxa_juros']].values )
## Min-max scaler
X_test['dias_atraso_dt_venc'] = mms.fit_transform( X_test[['dias_atraso_dt_venc']].values )
X_test['divida_atual'] = mms.fit_transform( X_test[['divida_atual']].values )
X_test['taxa_utilizacao_credito'] = ss.fit_transform( X_test[['taxa_utilizacao_credito']].values )

# Encodar var resposta de treino
d_class = {
    'Negar': 0,
    'Conceder': 1
}

y_test = y_test.map(d_class)

In [37]:
# Treinar nova XGBoost, com o novo dataset treino
# Model definition
model_xgboost_full = XGBClassifier(random_state=42)
# Model training
model_xgboost_full.fit(x_train_full, y_train_full)
# Model prediction
# Passar os dados de teste separados anteriormente
yhat_xgboost_full = model_xgboost_full.predict(X_test)

In [38]:
# Validar modelo com F1 Score (dados teste)
print(f"XGBoost - Média ponderada com F1 Score (dados teste): {round(f1_score(y_test, yhat_xgboost_full, average='weighted') ,3)}") #binary=default

XGBoost - Média ponderada com F1 Score (dados teste): 0.844


In [ ]:
# Parou!!!

In [ ]:
# Carregar dataset teste
df_ktest_raw = pd.read_csv('../data/df_kaggle/test.csv')
x_ktest = df_ktest_raw.copy()
# Conferir dataset teste kaggle
x_ktest

In [ ]:
# Encodar features categóricas de teste kaggle
x_ktest['investe_exterior'] = le_inv_ext.transform(x_ktest['investe_exterior'])
x_ktest['pessoa_polit_exp'] = le_pes_pol.transform(x_ktest['pessoa_polit_exp'])

In [ ]:
# Fazer a previsão do score usando o modelo final treinado:
yhat_xgboost_ktest = model_xgboost_full.predict(x_ktest)

In [ ]:
# Preparação para submissão
# Criar dataset final
df_sub = pd.concat([df_ktest_raw['id_cliente'], pd.Series(yhat_xgboost_ktest, name='limite_adicional')], axis=1)

In [ ]:
# Traduzir var resposta
lim_adi_dic_final = {0 : 'Negar', 1: 'Conceder'}
df_sub['limite_adicional'] = df_sub['limite_adicional'].map( lim_adi_dic_final )

In [ ]:
# Visutalizar arquivo final para submissão ao Kaggle
df_sub[:5]

## Preparing submission

In [ ]:
%%time

df_test = pd.read_csv('data/test.csv')

#df_test['bank_account'] = df_test['bank_account'].map(d_class) #mapeamento class

df_test['investe_exterior'] = df_test['investe_exterior'].map(lambda x: 0.76 if x == 0.15 else 0.24)
df_test['pessoa_polit_exp'] = df_test['pessoa_polit_exp'].map(lambda x: 0.94 if x == 0.16 else 0.06)

df_test['taxa_utilizacao_credito'] = ss.fit_transform( df_test[['taxa_utilizacao_credito']].values )

rs = RobustScaler()
mms = MinMaxScaler()

## Robust Scaler
# idade
df_test['idade'] = rs.fit_transform( df_test[['idade']].values )

# saldo_atual
df_test['saldo_atual'] = rs.fit_transform( df_test[['saldo_atual']].values )

# renda_anual
df_test['renda_anual'] = rs.fit_transform( df_test[['renda_anual']].values )

# valor_em_investimentos
df_test['valor_em_investimentos'] = rs.fit_transform( df_test[['valor_em_investimentos']].values )

# num_emprestimos
df_test['num_emprestimos'] = rs.fit_transform( df_test[['num_emprestimos']].values )

# num_contas_bancarias
df_test['num_contas_bancarias'] = rs.fit_transform( df_test[['num_contas_bancarias']].values )

# num_cartoes_credito
df_test['num_cartoes_credito'] = rs.fit_transform( df_test[['num_cartoes_credito']].values )

# num_pgtos_atrasados
df_test['num_pgtos_atrasados'] = rs.fit_transform( df_test[['num_pgtos_atrasados']].values )

# num_consultas_credito
df_test['num_consultas_credito'] = rs.fit_transform( df_test[['num_consultas_credito']].values )

# taxa_juros
df_test['taxa_juros'] = rs.fit_transform( df_test[['taxa_juros']].values )

## MinMax Scaler
# dias_atraso_dt_venc
df_test['dias_atraso_dt_venc'] = mms.fit_transform( df_test[['dias_atraso_dt_venc']].values )

# divida_atual
df_test['divida_atual'] = mms.fit_transform( df_test[['divida_atual']].values )

In [ ]:
#df_ss = pd.read_csv('/kaggle/input/inclusao-financeira-na-africa/test.csv')
preds = rf.predict(df_test.drop(columns='id_cliente'))

In [ ]:
preds

In [ ]:
df_submission = pd.DataFrame()
df_submission['id_cliente'] = df_test['id_cliente']
df_submission['limite_adicional'] = preds
df_submission['limite_adicional'] = df_submission['limite_adicional'].map(lambda x: 'Conceder' if x == 1 else 'Negar')
df_submission = df_submission.set_index('id_cliente')
df_submission.head()

In [ ]:
df_submission.to_csv('data/submissions/ciclo5_v2.csv')

In [ ]:
### Ideias de melhorias
    ### 1. Testar novas formas de encoding (1d)
    ### 2. Feature engineering (2d)
    ### 3. Hyper-parameter tuning (1d)
    ### 4. Combinação de modelos